In [ ]:
%cd /home/smalani/Cybernetic

from growth import parFun as growth_parFun
from growth import odeFun as growth_odeFun
import numpy as np

from gardner_opto import pulse_integrator
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import root_scalar

from NumbaMinpack import lmdif, hybrd, minpack_sig
from numba import njit, cfunc
import numpy as np

In [ ]:
# System of equations must look like this. Returns nothing!
@cfunc(minpack_sig)
def myfunc(var, fvec, args):
    x = var[0]
    tau_off = args[0]
    tau_on = args[1]
    n = args[2]
    var_dark = args[3]
    fvec[0] = (x ** n) / (x ** n + tau_on ** n) - var_dark
    
funcptr = myfunc.address # address in memory to myfunc

x_init = np.array([2]) # initial conditions
neqs = 1 # number of equations

tau_off=30
tau_on=5
n=4

args = (tau_off, tau_on, n, 0.3) # data you want to pass to myfunc

xsol, fvec, success, info = hybrd(funcptr, x_init, args) # OR solve with hybrd

print(xsol)
print(success)
print(info)

@njit
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

@cfunc(minpack_sig)
def myf_numba(var, fvec, args):
    tau_off=30
    tau_on=5
    n=4

    pulse_on = args[0]
    pulse_period = args[1]
    pulse_off = pulse_period - pulse_on
    x = sigmoid(float(var[0]))

    # x_init = np.array([2]) # initial conditions
    neqs = 1 # number of equations
    args = (tau_off, tau_on, n, x) # data you want to pass to myfunc

    solved = False
    
    # print(var)
    while not solved:
        x_init = np.array([np.random.rand() * pulse_period])
        xsol, _, success, _ = hybrd(funcptr, x_init, args) # OR solve with hybrd
        if xsol > 0 and success is True:
            solved = True
    t_status = xsol[0]

    var_light = (pulse_on + t_status) ** n / ((pulse_on + t_status) ** n + tau_on ** n)

    var_dark = var_light * np.exp(-(pulse_off / tau_off))
    
    fvec[0] = var_dark - x

funcptr2 = myf_numba.address # address in memory to myfunc

u = 1.
period = 9.9
args = (u*period, period) # data you want to pass to myf_numba
x_init = np.array([0.2]) # initial conditions
xsol, fvec, success, info = hybrd(funcptr2, x_init, args) # OR solve with hybrd

print(sigmoid(xsol))
print(success)
print(info)

In [ ]:
from scipy.optimize import root
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def myfunc_normalpy(var, args):
    x = var[0]
    tau_off = args[0]
    tau_on = args[1]
    n = args[2]
    var_dark = args[3]
    return (x ** n) / (x ** n + tau_on ** n) - var_dark

def myf_normalpy(var, args):
    tau_off=30
    tau_on=5
    n=4

    pulse_on = args[0]
    pulse_period = args[1]
    pulse_off = pulse_period - pulse_on
    x = sigmoid(var[0])

    # x_init = np.array([2]) # initial conditions
    neqs = 1 # number of equations
    args = (tau_off, tau_on, n, x) # data you want to pass to myfunc

    solved = False
    
    # print(var)
    while not solved:
        x_init = np.array([np.random.rand() * pulse_period])
        sol = root(myfunc_normalpy, x_init, args=(args,), method='hybr')
        # xsol, _, success, _ = hybrd(funcptr, x_init, args) # OR solve with hybrd
        if sol.x > 0 and sol.success is True:
            solved = True
    t_status = sol.x[0]

    var_light = (pulse_on + t_status) ** n / ((pulse_on + t_status) ** n + tau_on ** n)

    var_dark = var_light * np.exp(-(pulse_off / tau_off))
    
    return var_dark - x




u = 0.9
period = 8.3
args = (u*period, period) # data you want to pass to myf_numba
x_init = np.array([0.2]) # initial conditions
sol = root(myf_normalpy, x_init, args=(args,), method='hybr')

print(sigmoid(sol.x))
print(sol.x)
print(sol.success)
print(sol.message)

In [ ]:
myf_numba(x_init,[2],args)

In [ ]:
def trans_mean_func(u, period, guess=None):
    # if guess is None:
    #     sol = root_scalar(myf_small, bracket=[0,1], args=(period*u,period))
    # else:
    #     sol = root_scalar(myf_small, bracket=[0,1], args=(period*u,period), x0 = guess)
    x_init = np.array([0.2]) # initial conditions
    args = (u*period, period) # data you want to pass to myf_numba
    xsol, fvec, success, info = hybrd(funcptr2, x_init, args) # OR solve with hybrd
    xsol = sigmoid(xsol)

    if u > 0:
        t_step = min(period/1000,period*u)
    else:
        t_step = period/1000
    t_out, act_out, light_out, light_time_out = pulse_integrator(pulse_on=u*period, 
                                                             pulse_period=period, 
                                                             t_start=0, 
                                                             t_end=period, 
                                                             act_init=xsol, 
                                                             t_step=t_step)

    act_mean = np.trapz(act_out, t_out) / period
    return act_mean



u = 0.01
period = 100


print(trans_mean_func(u, period))
    

In [ ]:
alpha_e= 0.01 /1
alpha_star_e = 0.001 /1
beta_e = 0.002 /1

alpha_p = 0.01
beta_p = 0.0002
tau_p = 1 / beta_e

K_p = 0.1

t_frac = 1

# u_arr = [0.1, 0.9, 0.1]
# t_arr = [0, 100, 5000, 10000]

u_arr = [0.1, 0.3, 0.9, 0.6, 0.1, 0.9]
t_arr = [0, 100, 3000, 6000, 9000, 12000, 15000]

def my_u_fun(t):
    for i in range(len(t_arr)):
        if t < t_arr[-1-i]:
            continue
        else:
            return u_arr[-i]

def steady_state(act_mean):
    e0 = (alpha_e*act_mean + alpha_star_e) / beta_e
    # p0 = alpha_p * e0 / beta_p
    # p0 = e0 * alpha_p / (beta_p * (K_p + e0))
    p0 = (alpha_e*act_mean + alpha_star_e) / beta_e
    E0 = e0 * beta_e / (alpha_e + alpha_star_e)
    return [E0, p0]

def enzyme_fun(t, vars, alpha_fun):
    E, p = vars

    e = E * (alpha_e + alpha_star_e) / beta_e


    dedt = alpha_e * alpha_fun(t) - beta_e * e + alpha_star_e
    # dpdt = alpha_p * e - beta_p * p

    dEdt = dedt * beta_e / (alpha_e + alpha_star_e)
    dpdt = dEdt

    return [dEdt, dpdt]

def enzyme_fun_light(t, vars, alpha_fun, u_fun, period):
    print(t)
    E, p = vars

    e = E * (alpha_e + alpha_star_e) / beta_e

    
    dedt = alpha_e * alpha_fun(t) - beta_e * e + alpha_star_e
    # dpdt = alpha_p * e - beta_p * p

    dEdt = dedt * beta_e / (alpha_e + alpha_star_e)

    global e_sp
    act_mean = trans_mean_func(u_fun(t), period, guess=e_sp)
    e_sp, _ = steady_state(act_mean)
    dpdt = (e_sp - p) / tau_p
    # dpdt = dEdt

    return [dEdt, dpdt]

print(steady_state(0))



In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, root_scalar


t_span = [t_arr[0],t_arr[1]]
u = u_arr[0]

# sol = root_scalar(myf_small, bracket=[0,1], args=(period*u,period))
args = (u*period, period) # data you want to pass to myf_numba
x_init = np.array([0.2]) # initial conditions
xsol, fvec, success, info = hybrd(funcptr2, x_init, args) # OR solve with hybrd
xsol = sigmoid(xsol)

t_out_arr, act_out_arr, light_out_arr, light_time_out_arr = [], [], [], []
if u > 0:
    t_step = min(period/1000,period*u)
else:
    t_step = period/1000
t_out, act_out, light_out, light_time_out = pulse_integrator(pulse_on=u*period, 
                                                             pulse_period=period, 
                                                             t_start=t_span[0], 
                                                             t_end=t_span[1], 
                                                             act_init=xsol, 
                                                             t_step=t_step)
t_out_arr.append(t_out)
act_out_arr.append(act_out)
light_out_arr.append(light_out)
light_time_out_arr.append(light_time_out)

print('shapity shape')
period_len = int(act_out.size / (t_span[1] / period))
act_mean = np.trapz(act_out[-period_len:], t_out[-period_len:],) / period
print(act_mean)

for i in range(len(u_arr) - 1):
    t_span = np.array([t_arr[i+1],t_arr[i+2]])
    u = u_arr[i+1]
    if u > 0:
        t_step = min(period/1000,period*u)
    else:
        t_step = period/1000
    u0 = act_out[-1]
    t_out, act_out, light_out, light_time_out = pulse_integrator(pulse_on=u*period, 
                                                                pulse_period=period, 
                                                                t_start=t_span[0], 
                                                                t_end=t_span[1], 
                                                                act_init=act_out[-1], 
                                                                t_step=t_step)
    t_out_arr.append(t_out)
    act_out_arr.append(act_out)
    light_out_arr.append(light_out)
    light_time_out_arr.append(light_time_out)

t_out = np.concatenate(t_out_arr)
act_out = np.concatenate(act_out_arr)
light_out = np.concatenate(light_out_arr)
light_time_out = np.concatenate(light_time_out_arr)

alpha1_fun = interp1d(t_out, act_out)
zero_line = np.zeros(light_time_out.shape)



# e0 = [(alpha*u0 + alpha_star) / beta]
# e0 = (alpha_e*act_mean + alpha_star_e) / beta_e
# p0 = alpha_p * e0 / beta_p
# e0 = [400]
e0, p0 = steady_state(act_mean)

t_span = [0,t_span[1]]

# sol = solve_ivp(enzyme_fun, t_span, [e0, e0], args=(alpha1_fun,), 
#                     rtol=1e-12, atol=1e-15, 
#                     method='BDF',
#                     first_step=0.1,
#                 )

e_sp = None

sol = solve_ivp(enzyme_fun_light, t_span, [e0, e0], args=(alpha1_fun,my_u_fun,period), 
                    rtol=1e-6, atol=1e-9, 
                    method='BDF',
                    first_step=0.1,
                )
    

In [ ]:
print(e0)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(t_out/60/60, act_out, linewidth=3)
ax.fill_between(light_time_out/60/60, light_out, zero_line, color='cyan', alpha=0.2)
ax.set_ylabel('Transcription', fontsize=25)
ax.set_xlabel('Time', fontsize=25)
ax.tick_params(axis='both', labelsize =20)
# ax.set_yticks(fontsize=20)

ax2 = ax.twinx()
ax2.plot(sol.t/60/60, sol.y[0], c='orange', linewidth=3)
ax2.plot(sol.t/60/60, sol.y[1], c='green', linewidth=3)
# ax2.set_ylim([-0.1,1.1])
ax2.tick_params(axis='both', labelsize =20)

In [ ]:
ratio = 0.1

indexing = int(ratio * sol.t.size)
indexing2 = int(ratio * t_out.size)
indexing3 = int(ratio *  light_time_out.size)

fig, ax = plt.subplots(constrained_layout=True)
ax.plot(t_out[:indexing2]/60/60, act_out[:indexing2], linewidth=3)
ax.fill_between(light_time_out[:indexing3]/60/60, light_out[:indexing3], zero_line[:indexing3], color='cyan', alpha=0.2)
ax.set_ylabel('Transcription', fontsize=25)
ax.set_xlabel('Time', fontsize=25)
ax.tick_params(axis='both', labelsize =20)
# ax.set_yticks(fontsize=20)

ax2 = ax.twinx()
ax2.plot(sol.t[:indexing]/60/60, sol.y[0][:indexing], c='orange', linewidth=3)
ax2.plot(sol.t[:indexing]/60/60, sol.y[1][:indexing], c='green', linewidth=3)
# ax2.set_ylim([-0.1,1.1])
ax2.tick_params(axis='both', labelsize =20)

In [ ]:
print(period)
print(beta_e)